In [3]:
import numpy as np
import os
import matplotlib
import pandas as pd

In [4]:
#read in whatever datafile you want
def read_datafunc(datafile):
    read_data = pd.read_csv(datafile)
    return read_data

In [41]:
#summarize the size and columns, of the data
def summarize_data(datafile):
    read_data = read_datafunc(datafile)
    print(datafile)
    print('Dimensions:', read_data.shape)
    print('Columns:', list(read_data.columns))
    return read_data
    

df = summarize_data('../data/ind_salud_v0.csv')

../data/ind_salud_v0.csv
Dimensions: (2868, 10)
Columns: ['idindicador', 'idunidad', 'iddepto', 'nomdepto', '2005', '2006', '2007', '2008', '2009', '2010']


In [6]:
#removing columns that have highly sparse data
sparsity_threshold = .5
df_nonsparse_columns = df.loc[:,(df.isnull().sum()/len(df)) < sparsity_threshold]

#look at the new shape of the dataset
print(df_nonsparse_columns.shape)
print(df_nonsparse_columns.columns)

(2868, 10)
Index(['idindicador', 'idunidad', 'iddepto', 'nomdepto', '2005', '2006',
       '2007', '2008', '2009', '2010'],
      dtype='object')


In [88]:
#looking at new indicadors
# print(df_nonsparse_columns[['idindicador', 'nomindicador']].drop_duplicates())
# print(len(df_nonsparse_columns.idindicador.unique()))

    idindicador                                       nomindicador
0      ttdiesti               Tasa de mortalidad infantil estimada
1      ctrmormt                       Mortalidad materna a 42 días
4      caelacex  Duración de la lactancia materna exclusiva en ...
6      pcrparin                  Porcentaje partos institucionales
24     rtrmorpu              Razón de mortalidad materna a 42 días
36     pbepredc                Prevalencia de desnutrición crónica
42     ptrldeng                          Letalidad de Dengue grave
45     pbretaia  Porcentaje de brotes de ETA con identificación...
55     pcrperca  Porcentaje de partos atendidos por personal ca...
57     pbehtaco  Porcentaje de población con hipertension arter...
62     igrtgfec                          Tasa Global de Fecundidad
63     pbrmivih  Porcentaje de transmisión materno infantil de VIH
69     ttdnesti   Tasa de mortalidad estimada en menores de 5 años
71     rgrtgfec                         Tasa General de Fecund

In [7]:
##drop all rows where yea2005:yea2010 are all empty
not_null = df_nonsparse_columns.dropna(subset=['2005','2006','2007','2008','2009','2010'], 
                                       how='all', thresh=2)

In [28]:
## looking at the new indicadores we kept
print(not_null.shape)
print(df_nonsparse_columns.shape)
indi = not_null[['idindicador']].drop_duplicates()
print(indi, len(indi))

(2626, 10)
(2868, 10)
    idindicador
0      ttdiesti
1      ctrmormt
6      pcrparin
24     rtrmorpu
36     pbepredc
42     ptrldeng
55     pcrperca
63     pbrmivih
69     ttdnesti
96     pcrrncpn
103    cgrnacim
247    pgeembar
300    paemetmo
671    caelacex
882    pbepredg 15


In [29]:
##group the dataframe by departamento and indicador
by_depto_indicador = not_null.groupby(by=['iddepto', 'idindicador'])

In [30]:
#keeping only yea2005:yea2010 columns find the average of the value by departamento

mean = by_depto_indicador[['2005','2006','2007','2008','2009','2010']].agg('mean')
median = by_depto_indicador[['2005','2006','2007','2008','2009','2010']].agg('median')

In [31]:
##ungrouping and setting index to be columns
median_reset = median.reset_index(level=['iddepto', 'idindicador'])

#melting to get into one row to see which indicadores are actually very sparse
median_melted = pd.melt(median_reset, id_vars=['iddepto', 'idindicador'], var_name='year')
median_melted_not_NA = pd.melt(median_reset, id_vars=['iddepto', 'idindicador'], var_name='year').dropna()

print(len(median_melted), len(median_melted_not_NA))

2256 1898


In [32]:
by_indicador_melted = median_melted.groupby('idindicador')
by_indicador_melted_not_NA = median_melted_not_NA.groupby('idindicador')

In [33]:
print(by_indicador_melted['idindicador'].value_counts().sort_values())
print(by_indicador_melted_not_NA['idindicador'].value_counts().sort_values())

idindicador  idindicador
caelacex     caelacex         6
paemetmo     paemetmo         6
pbepredg     pbepredg         6
pbepredc     pbepredc        78
pbrmivih     pbrmivih       150
ttdnesti     ttdnesti       174
ptrldeng     ptrldeng       186
pcrparin     pcrparin       204
pcrperca     pcrperca       204
pcrrncpn     pcrrncpn       204
pgeembar     pgeembar       204
rtrmorpu     rtrmorpu       204
ttdiesti     ttdiesti       204
cgrnacim     cgrnacim       210
ctrmormt     ctrmormt       216
Name: idindicador, dtype: int64
idindicador  idindicador
caelacex     caelacex         2
paemetmo     paemetmo         2
pbepredg     pbepredg         2
pbepredc     pbepredc        26
pbrmivih     pbrmivih        56
pgeembar     pgeembar        68
ptrldeng     ptrldeng       122
ttdnesti     ttdnesti       174
pcrparin     pcrparin       204
pcrperca     pcrperca       204
pcrrncpn     pcrrncpn       204
rtrmorpu     rtrmorpu       204
ttdiesti     ttdiesti       204
cgrnacim     cgrnacim 

In [34]:
keep = by_indicador_melted_not_NA['idindicador'].value_counts().sort_values()>150
print(keep)

idindicador  idindicador
caelacex     caelacex       False
paemetmo     paemetmo       False
pbepredg     pbepredg       False
pbepredc     pbepredc       False
pbrmivih     pbrmivih       False
pgeembar     pgeembar       False
ptrldeng     ptrldeng       False
ttdnesti     ttdnesti        True
pcrparin     pcrparin        True
pcrperca     pcrperca        True
pcrrncpn     pcrrncpn        True
rtrmorpu     rtrmorpu        True
ttdiesti     ttdiesti        True
cgrnacim     cgrnacim        True
ctrmormt     ctrmormt        True
Name: idindicador, dtype: bool


In [44]:
by_indicador_melted_not_NA.head()

,iddepto,idindicador,year,value
0,5.0,cgrnacim,2005,94636.000000
1,5.0,ctrmormt,2005,61.000000
2,5.0,pbepredc,2005,11.400000
4,5.0,pcrparin,2005,97.650000
5,5.0,pcrperca,2005,97.789425
6,5.0,pcrrncpn,2005,84.310000
7,5.0,pgeembar,2005,22.100000
9,5.0,rtrmorpu,2005,64.460000
10,5.0,ttdiesti,2005,18.315000
11,5.0,ttdnesti,2005,25.130432


In [45]:
#reading GIS Dataframe
GISdf = summarize_data('../GIS_data/country_code_overview - Sheet1.csv')

../GIS_data/country_code_overview - Sheet1.csv
Dimensions: (35, 8)
Columns: ['ID', 'OpenData', 'GADM', 'Department', 'Capital', 'Area (km²)', 'Population', 'Unnamed: 7']


In [55]:
GISdf.head()

,ID,OpenData,GADM,Department,Capital,Area (km²),Population,Unnamed: 7
0,0.0,11.0,Non existent,Capital District,Bogotá,"1,587","8,254,722",Part of cundinamarca?
1,1.0,91.0,1,Amazonas,Leticia,"109,665","80,360",NaN
2,2.0,5.0,2,Antioquia,Medellín,"63,612","5,750,478",NaN
3,3.0,81.0,3,Arauca,Arauca,"23,818","282,302",NaN
4,4.0,8.0,4,Atlántico,Barranquilla,"3,388","2,365,663",NaN


In [61]:
pd.DataFrame.merge(GISdf[['OpenData', 'GADM', 'Department']], median_melted_not_NA, on=GISdf['OpenData'])

KeyError: '[  11.   91.    5.   81.    8.   13.   15.   17.   18.   85.   19.   20.\n   27.   23.   25.   94.   95.   41.   44.   47.   50.   52.   54.   86.\n   63.   66.   91.   68.   70.   73.   76.   97.   99.   nan  170.] not in index'